In [ ]:
%matplotlib inline


# Sensitivity analysis of DEXP to noise level

This code shows a step-by-step processing of potential field imaging aiming at giving an estimate of electrical sources positions and depth using the dEXP tranformation method.
dEXP method implementation from Fedi et al. 2012. 
Calculations used :mod:`dEXP`, while plotting use the :mod:`plot_dEXP` module.

Application on a anomaly of electrical resistivity.
The model data was created using geometric objects from :mod:`pygimli.meshtools`. The forward simulation of the data was done using :mod:`pygimli.ERTsimulate` module.
The noise is added to the data. Here 1% plus 1µV. Note, we force a specific noise seed as we want reproducable results for testing purposes.

<div class="alert alert-info"><h4>Note</h4><p>This is part of a larger project aiming at inverting current sources density (see more at: https://icsd-dev.readthedocs.io/en/latest/)</p></div>


**References**

Uieda, L., V. C. Oliveira Jr, and V. C. F. Barbosa (2013), Modeling the Earth with Fatiando a Terra, Proceedings of the 12th Python in Science Conference, pp. 91 - 98.

Uieda, L. (2018). Verde: Processing and gridding spatial data using Green's functions. Journal of Open Source Software, 3(29), 957. doi:10.21105/joss.00957

Fedi, M., and M. Pilkington (2012), Understanding imaging methods for potential
field data, Geophysics, 77(1), G13, doi:10.1190/geo2011-0078.1

Rucker, C., Gunther, T., Wagner, F.M., 2017. pyGIMLi: An open-source library for modelling and inversion in geophysics, Computers and Geosciences, 109, 106-123, doi: 10.1016/j.cageo.2017.07.011




In [ ]:
from fatiando.vis.mpl import square
from fatiando import gridder

# my own functions
import lib.dEXP as dEXP
import lib.plot_dEXP as pEXP
import lib.set_parameters as para

# exemples
import examples.malm.loadmalm.Load_sens_MALM as MALM

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 15

Code steps to reproduce:
- Load datasets
- Set DEXP parameters (ridges min and max, derivative order, ...)
- Potential field smoothing
- Upward continuation of the field data
- Ridges identification
- Filter ridges (regionally constrainsted)
- Fit ridges (regionally constrainsted)
- Plot ridges (regionally constrainsted)



In [ ]:
MESH = []
LABEL = []
DF_F = []
DF_FIT = []
XXZZ = []
CTm = []


filenames = ['MSoilR1000.0AnoR1Z-13.75W5H2.5L5Noise0',
             'MSoilR1000.0AnoR1Z-13.75W5H2.5L5Noise0.01', # 10% of noise
             'MSoilR1000.0AnoR1Z-13.75W5H2.5L5Noise0.01', # use smooth fct
             'MSoilR1000.0AnoR1Z-13.75W5H2.5L5Noise0.01', # use derivative order 1
             'MSoilR1000.0AnoR1Z-13.75W5H2.5L5Noise0.01'] # use derivative order 2

for i, fi in enumerate(filenames):
    (x_raw, y_raw, z_raw, U_raw, 
     maxdepth, shape_raw, 
     p1, p2, 
     SimName, ano_prop) = MALM.load_MALM_sens3d(filename='./loadmalm/' +
                                                                fi + '.pkl')
    shape = (150,150)
    # print(U_raw[1:10])
    xp,yp,U = gridder.interp(x_raw,y_raw,U_raw,shape)
    
    
    parameters = para.set_par(shape=shape,max_elevation=abs(maxdepth))
    interp = True
    scaled = parameters[0]
    SI = parameters[1]
    zp, qorder, nlay = parameters[2:5]
    minAlt_ridge, maxAlt_ridge = parameters[5:7]
    fix_peak_nb = 2
    #%%
    # ridges analysis parameters
    nlay = 25
    max_elevation = 20
    minAlt_ridge = max_elevation*0.25
    maxAlt_ridge = max_elevation*0.75
    
    interp = True
    smooth = False 
    x_axis = 'y'

    #%%
    # Anomalies properties
    # HDWL : height, Depth, Width (x), Lenght (y)
    x1, x2, z1, z2 = [max(x_raw)/2-ano_prop['HWDL'][1]/2,max(x_raw)/2 + ano_prop['HWDL'][1]/2,
                    ano_prop['HWDL'][2]+ ano_prop['HWDL'][0]/2,
                    ano_prop['HWDL'][2]- ano_prop['HWDL'][0]/2]
    xxzz = [x1, x2, z1, z2]
    CT = ano_prop['SoilR']/ano_prop['AnoR']
    
    #%% 
    # some constrainsts
    if i==2:
    # smooth the data 
        U = dEXP.smooth2d(xp, yp, U, sigma=1)
        # smooth = True 

    if i==3:
    # inscrease derivative order 
        qorder = qorder + 1
        # fix_peak_nb = 4
        U = dEXP.smooth2d(xp, yp, U, sigma=1)
    
    if i==4:
    # inscrease derivative order 
        qorder = qorder + 1
        
    #%% 
    # Plot the data 
    # pEXP.plot_line(xp, yp, U,p1,p2, interp=interp)
    
    #%% 
    # Pad the edges of grids (if necessary)
    # xp,yp,U, shape = dEXP.pad_edges(xp,yp,U,shape,pad_type=0) # reflexion=5
    # pEXP.plot_line(xp, yp,U,p1,p2, interp=interp)
    
    
    #%% 
    # Upward continuation of the field data

    mesh, label_prop = dEXP.upwc(xp, yp, zp, U, shape, 
                     zmin=0, zmax=max_elevation, nlayers=nlay, 
                     qorder=qorder)
    
    


    
    # plt, cmap = pEXP.plot_xy(mesh, label=label_prop,Xaxis=x_axis)
    # plt.colorbar(cmap, label=label_prop)
    
    #%%
    # Ridges identification
    # dEXP.ridges_minmax_plot(xp, yp, mesh, p1, p2,
    #                                       label=label_prop,
    #                                       fix_peak_nb=2,
    #                                       method_peak='find_peaks')  
    
    # or  find_peaks or peakdet or spline_roots
    dfI,dfII, dfIII, ax = dEXP.ridges_minmax(xp, yp, mesh, p1, p2,
                                          label=label_prop,
                                          fix_peak_nb=fix_peak_nb,
                                          method_peak='find_peaks',
                                          smooth=smooth,
                                          Xaxis=x_axis,
                                          showfig=True
                                          )  
    
    ax.savefig('upwc_noise_s' + str(i) + '.svg')
    ax.savefig('upwc_noise_s' + str(i) + '.png', dpi=450)
    ax.savefig('upwc_noise_s' + str(i) + '.eps')
    
    
    #%%
    # Filter ridges (regionally constrainsted)
    
    dfI_f,dfII_f, dfIII_f = dEXP.filter_ridges(dfI,dfII,dfIII,
                                                minDepth=minAlt_ridge,
                                                maxDepth=maxAlt_ridge,
                                                minlength=7,rmvNaN=True,
                                                xmin=100, xmax=300,
                                                Xaxis=x_axis
                                                )
    df_f = dfI_f, dfII_f, dfIII_f     
        
    #%%
    # fit 
    df_fit = dEXP.fit_ridges(df_f, rmvOutliers=True) # fit ridges on filtered data
    
    #%% 
    # save data loop
    MESH.append(mesh)
    LABEL.append(label_prop)
    DF_F.append(df_f)
    DF_FIT.append(df_fit)
    XXZZ.append(xxzz)
    CTm.append(CT)

<div class="alert alert-info"><h4>Note</h4><p>0% of noise</p></div>



In [ ]:
i = 0
fig, ax = plt.subplots(figsize=(15,3))
# ax = plt.gca()
pEXP.plot_xy(MESH[i], label=LABEL[i], ax=ax) #, ldg=)
dfI_f,dfII_f,dfIII_f = DF_F[i]
pEXP.plot_ridges_harmonic(dfI_f,dfII_f,dfIII_f,ax=ax,label=False,legend=False)   
pEXP.plot_ridges_sources(DF_FIT[i], ax=ax, z_max_source=-max_elevation*1.2,
                          ridge_type=[0,1,2],ridge_nb=None)
# plt.xlim([100,300])
x1, x2, z1, z2 = XXZZ[i]
square([x1, x2, z1, z2])
plt.annotate(CTm[i],[(x1 + x2)/2, (z1+z2)/2])
plt.title('0% of noise')
plt.savefig('noise_s0.svg')
plt.savefig('noise_s0.png', dpi=450)
plt.savefig('noise_s0.eps')

<div class="alert alert-info"><h4>Note</h4><p>10% of noise</p></div>



In [ ]:
i = 1
fig, ax = plt.subplots(figsize=(15,3))
pEXP.plot_xy(MESH[i], label=LABEL[i], ax=ax) #, ldg=)
dfI_f,dfII_f,dfIII_f = DF_F[i]
pEXP.plot_ridges_harmonic(dfI_f,dfII_f,dfIII_f,ax=ax,label=False)   
pEXP.plot_ridges_sources(DF_FIT[i], ax=ax, z_max_source=-max_elevation*1.2,
                          ridge_type=[0,1,2],ridge_nb=None)
x1, x2, z1, z2 = XXZZ[i]
square([x1, x2, z1, z2])
plt.annotate(CTm[i],[(x1 + x2)/2, (z1+z2)/2])
plt.title('10% of noise')
# plt.tight_layout()
plt.savefig('noise_s1.svg')
plt.savefig('noise_s1.png', dpi=450)
plt.savefig('noise_s1.eps')

<div class="alert alert-info"><h4>Note</h4><p>10% of noise - smoothed</p></div>



In [ ]:
i = 2
fig, ax = plt.subplots(figsize=(15,3))
pEXP.plot_xy(MESH[i], label=LABEL[i], ax=ax) #, ldg=)
dfI_f,dfII_f,dfIII_f = DF_F[i]
pEXP.plot_ridges_harmonic(dfI_f,dfII_f,dfIII_f,ax=ax,label=False)   
pEXP.plot_ridges_sources(DF_FIT[i], ax=ax, z_max_source=-max_elevation*1.2,
                          ridge_type=[0,1,2],ridge_nb=None)
x1, x2, z1, z2 = XXZZ[i]
square([x1, x2, z1, z2])
plt.annotate(CTm[i],[(x1 + x2)/2, (z1+z2)/2])
plt.title('10% of noise - smoothed')
# plt.tight_layout()
plt.savefig('noise_s2.svg')
plt.savefig('noise_s2.png', dpi=450)
plt.savefig('noise_s2.eps')

# .. note::
#
#     10% of noise - smoothed - q-order=1

i = 3
fig, ax = plt.subplots(figsize=(15,3))
pEXP.plot_xy(MESH[i], label=LABEL[i], ax=ax) #, ldg=)
dfI_f,dfII_f,dfIII_f = DF_F[i]
pEXP.plot_ridges_harmonic(dfI_f,dfII_f,dfIII_f,ax=ax,label=False)   
pEXP.plot_ridges_sources(DF_FIT[i], ax=ax, z_max_source=-max_elevation*1.2,
                          ridge_type=[0,1,2],ridge_nb=None)
x1, x2, z1, z2 = XXZZ[i]
square([x1, x2, z1, z2])
plt.annotate(CTm[i],[(x1 + x2)/2, (z1+z2)/2])
plt.title('10% of noise - q-order=1')
plt.savefig('noise_s3.svg')
plt.savefig('noise_s3.png', dpi=450)
plt.savefig('noise_s3.eps')



i = 4
fig, ax = plt.subplots(figsize=(15,3))
pEXP.plot_xy(MESH[i], label=LABEL[i], ax=ax) #, ldg=)
dfI_f,dfII_f,dfIII_f = DF_F[i]
pEXP.plot_ridges_harmonic(dfI_f,dfII_f,dfIII_f,ax=ax,label=False)   
pEXP.plot_ridges_sources(DF_FIT[i], ax=ax, z_max_source=-max_elevation*1.2,
                          ridge_type=[0,1,2],ridge_nb=None)
x1, x2, z1, z2 = XXZZ[i]
square([x1, x2, z1, z2])
plt.annotate(CTm[i],[(x1 + x2)/2, (z1+z2)/2])
plt.title('10% of noise - q-order=1')
plt.savefig('noise_s4.svg')
plt.savefig('noise_s4.png', dpi=450)
plt.savefig('noise_s4.eps')